# Working with Missing Data

**Utilizing the California Housing dataset**

In [77]:
# Import package dependencies
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from ml_metrics import rmse
import matplotlib.pyplot as plt
from sklearn import datasets

In [78]:
# Load in the dataset
california = datasets.fetch_california_housing()
print(california.data.shape)

(20640, 8)


In [79]:
print(california.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR'])


In [80]:
print(california.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [81]:
california.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [82]:
# Convert the matrix to pandas
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,4.526,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,3.585,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,3.521,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,3.413,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.422,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [83]:
cal.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,2.068558,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.153956,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.149990,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,1.196000,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,1.797000,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,2.647250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,5.000010,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


## Start by fitting a Linear Regression model to the full dataset

**Create a training and testing split (ex., 70/30-split)**

In [84]:
# Create training and testing sets (cross-validation not needed)
train_set = cal.sample(frac=0.7, random_state=100)
test_set = cal[~cal.isin(train_set)].dropna()
print(train_set.shape[0])
print(test_set.shape[0])

14448
6192


In [85]:
train_set.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
8151,2.453,36.0,6.276836,1.039548,444.0,2.508475,33.81,-118.10
53,1.042,52.0,4.075000,1.140000,1162.0,2.905000,37.82,-122.27
3039,1.462,13.0,6.746647,1.062593,2170.0,3.233979,35.37,-119.12
9484,1.542,19.0,6.750000,1.348684,424.0,2.789474,39.31,-123.15
9307,3.242,31.0,4.477459,1.073087,2962.0,2.023224,37.98,-122.52


In [86]:
# Get the training and testing row indices for later use
train_index = train_set.index.values.astype(int)
test_index = test_set.index.values.astype(int)

In [87]:
# Demonstration of using the row indices above to select consistent records
cal.iloc[train_index].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
8151,2.453,36.0,6.276836,1.039548,444.0,2.508475,33.81,-118.10
53,1.042,52.0,4.075000,1.140000,1162.0,2.905000,37.82,-122.27
3039,1.462,13.0,6.746647,1.062593,2170.0,3.233979,35.37,-119.12
9484,1.542,19.0,6.750000,1.348684,424.0,2.789474,39.31,-123.15
9307,3.242,31.0,4.477459,1.073087,2962.0,2.023224,37.98,-122.52


In [88]:
# Converting the training and testing datasets back to matrix-formats
X_train = train_set.iloc[:, 1:].values # returns the data; excluding the target
Y_train = train_set.iloc[:, 0].values # returns the target-only
X_test = test_set.iloc[:, 1:].values # ""
Y_test = test_set.iloc[:, 0].values # ""

In [89]:
# Fit a linear regression to the training data
reg = LinearRegression(normalize=True).fit(X_train, Y_train)
print(reg.score(X_train, Y_train))
print(reg.coef_)
print(reg.intercept_)
print(reg.get_params())

0.4021408992488681
[ 5.39413657e-03  3.64187452e-01 -1.38129627e+00 -1.17589393e-05
 -1.24751112e-03 -7.30902386e-01 -7.23704703e-01]
-59.014861272508526
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True}


In [90]:
# Find the variable with the largest "normalized" coefficient value
print('The positive(max) coef-value is {}'.format(max(reg.coef_))) # Positive Max
#print('The abs(max) coef-value is {}'.format(max(reg.coef_, key=abs))) # ABS Max
max_var = max(reg.coef_) # Positive Max
#max_var = max(reg.coef_, key=abs) # ABS Max
var_index = reg.coef_.tolist().index(max_var)
print('The variable associated with this coef-value is {}'.format(california.feature_names[var_index]))

The positive(max) coef-value is 0.36418745206851905
The variable associated with this coef-value is HouseAge


In [91]:
Y_pred = reg.predict(X_test)

orig_mae = mean_absolute_error(Y_test,Y_pred)
orig_mse = mean_squared_error(Y_test,Y_pred)
orig_rmse_val = rmse(Y_test,Y_pred)
orig_r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%orig_mae)
print("MSE:  %.3f"%orig_mse)
print("RMSE:  %.3f"%orig_rmse_val)
print("R2:  %.3f"%orig_r2)

MAE: 0.678
MSE:  0.808
RMSE:  0.899
R2:  0.390


In [92]:
res_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'rmse':orig_rmse_val, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'rmse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])

In [93]:
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,0.678033,0.80827,0.899039,0.390038,NaN,NaN,NaN,NaN


## Round 1 of Imputation

**Here we can randomly sample the full dataset and replace a single column's values**

In [94]:
in_sample = cal.sample(frac=0.3, random_state=99)
in_sample.shape

(6192, 8)

In [95]:
out_sample = cal[~cal.isin(in_sample)].dropna()
out_sample.shape

(14448, 8)

In [96]:
print(out_sample.shape[0] + in_sample.shape[0])
print(cal.shape[0])

20640
20640


In [97]:
in_sample.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
7914,1.201,30.0,3.832661,1.046371,2685.0,5.413306,33.88,-118.08
11963,0.920,34.0,6.185930,1.085427,841.0,4.226131,34.01,-117.41
18738,0.852,12.0,4.513889,0.998843,2145.0,2.482639,40.56,-122.35
17431,1.353,27.0,4.203036,1.096774,1544.0,2.929791,34.65,-120.45
17947,2.769,35.0,5.109131,0.915367,1054.0,2.347439,37.33,-121.96


## Choose a variable to replace

In [98]:
in_sample['HouseAge'] = np.nan
in_sample.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
7914,1.201,NaN,3.832661,1.046371,2685.0,5.413306,33.88,-118.08
11963,0.920,NaN,6.185930,1.085427,841.0,4.226131,34.01,-117.41
18738,0.852,NaN,4.513889,0.998843,2145.0,2.482639,40.56,-122.35
17431,1.353,NaN,4.203036,1.096774,1544.0,2.929791,34.65,-120.45
17947,2.769,NaN,5.109131,0.915367,1054.0,2.347439,37.33,-121.96


**Choose an imputation method**

In [99]:
out_sample['HouseAge'].median()

29.0

In [100]:
in_sample['HouseAge'] = in_sample['HouseAge'].fillna(out_sample['HouseAge'].median())
in_sample.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
7914,1.201,29.0,3.832661,1.046371,2685.0,5.413306,33.88,-118.08
11963,0.920,29.0,6.185930,1.085427,841.0,4.226131,34.01,-117.41
18738,0.852,29.0,4.513889,0.998843,2145.0,2.482639,40.56,-122.35
17431,1.353,29.0,4.203036,1.096774,1544.0,2.929791,34.65,-120.45
17947,2.769,29.0,5.109131,0.915367,1054.0,2.347439,37.33,-121.96


**Rejoin the imputed and original datasets**

In [101]:
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()
imputed_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,4.526,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,3.585,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,3.521,29.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,3.413,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.422,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


**Use the same training and testing indices to fit the model**

In [102]:
train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]
train_set.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
8151,2.453,29.0,6.276836,1.039548,444.0,2.508475,33.81,-118.10
53,1.042,52.0,4.075000,1.140000,1162.0,2.905000,37.82,-122.27
3039,1.462,13.0,6.746647,1.062593,2170.0,3.233979,35.37,-119.12
9484,1.542,19.0,6.750000,1.348684,424.0,2.789474,39.31,-123.15
9307,3.242,31.0,4.477459,1.073087,2962.0,2.023224,37.98,-122.52


In [103]:
# Converting the training and testing datasets back to matrix-formats
X_train = train_set.iloc[:, 1:].values # returns the data; excluding the target
Y_train = train_set.iloc[:, 0].values # returns the target-only
X_test = test_set.iloc[:, 1:].values # ""
Y_test = test_set.iloc[:, 0].values # ""

**Fit a new model to the imputed dataset**

In [104]:
reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

0.40182348716655325
[ 5.88625535e-03  3.61966590e-01 -1.37351052e+00 -1.68408451e-05
 -1.21655089e-03 -7.34807395e-01 -7.28147765e-01]
-59.41113202265889
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}


In [105]:
Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

MAE: 0.677
MSE:  0.808
RMSE:  0.899
R2:  0.390


In [106]:
temp_frame = pd.DataFrame({'data':'30% imputed',
                   'imputation':'MAR',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

In [107]:
res_frame = pd.concat([res_frame, temp_frame])
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,0.678033,0.808270,0.899039,0.390038,NaN,NaN,NaN,NaN
0,30% imputed,MAR,0.677494,0.807814,0.898785,0.390382,-0.000539,-0.000456,-0.000254,0.000344
